In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('datasets/taxi_zones(from data.gov).csv')
df.head()

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens
2,3,0.084341,MULTIPOLYGON (((-73.84792614099985 40.87134223...,0.000314,Allerton/Pelham Gardens,3,Bronx
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan
4,5,0.092146,MULTIPOLYGON (((-74.17421738099989 40.56256808...,0.000498,Arden Heights,5,Staten Island


In [3]:
df = df.loc[:, ['the_geom', 'zone', 'LocationID', 'borough']]
df.shape

(263, 4)

In [4]:
df.isna().sum()

the_geom      0
zone          0
LocationID    0
borough       0
dtype: int64

In [5]:
geom = df.loc[:, 'the_geom'].values
geom.shape

(263,)

In [6]:
lat = []  # 緯度
lon = []  # 經度

for i in geom:
    geom_str = i.strip('MULTIPOLYGON ((').strip('))').split()  # 去除頭尾多餘的字串和()，並將每一個值獨立
    coordinate = np.array([float(geom_str[-1]), float(geom_str[0])])  # 取出第1個值和最後一個值轉為float，組成一個獨立array，稍後添加到主要array
    
    # 刪除已經獨立的第1個值和最後一個值
    geom_str = np.delete(geom_str, 0)
    geom_str = np.delete(geom_str, -1)

    # 將每個值中多餘的逗號, 空格, ()移除，並轉為float
    geom_float = np.array([])
    for j in geom_str:
        geom_float = np.append(geom_float, float(j.strip(', ').strip('((').strip('))')))
    
    geom_float = list(geom_float)
    
    # 將先前獨立的array加回
    geom_float += list(coordinate)

    # 將經緯度分別存成list
    lat_temp = []
    lon_temp = []
    for k in range(len(geom_float)):
        if k % 2 == 0:
            lat_temp.append(float(geom_float[k]))
        else:
            lon_temp.append(float(geom_float[k]))
    
    lat.append(np.mean(lat_temp))
    lon.append(np.mean(lon_temp))


df['lat'] = lat
df['lon'] = lon
df = df.astype({'lat': 'str', 'lon': 'str'})
df = df.drop('the_geom', axis=1)
df.head()

,zone,LocationID,borough,lat,lon
0,Newark Airport,1,EWR,40.69024330172403,-74.17427027586203
1,Jamaica Bay,2,Queens,40.61216268077739,-73.81764292356883
2,Allerton/Pelham Gardens,3,Bronx,40.86429403840487,-73.84650986336355
3,Alphabet City,4,Manhattan,40.72385314970739,-73.97520908904802
4,Arden Heights,5,Staten Island,40.55667805421755,-74.1898026069764


In [7]:
df.dtypes

zone          object
LocationID     int64
borough       object
lat           object
lon           object
dtype: object

In [8]:
df.to_csv('organized_taxi_zone_lat_lon.csv', index=False)

In [9]:
df[df.LocationID.isin([1, 132, 138])]

,zone,LocationID,borough,lat,lon
0,Newark Airport,1,EWR,40.69024330172403,-74.17427027586203
136,JFK Airport,132,Queens,40.63906784736892,-73.78374464223006
145,LaGuardia Airport,138,Queens,40.774627080754335,-73.87282104019933
